Lets Create a question answer section using T5 or other model on eli5 dataset.

If we use smaller llm models they can give the answer of the question but failed to give answer in human manner.
I have pre tained the bert model but it will only provide the extraceted asnwer. By extracted it mean the exact words will be generated againt the question.

Lets make it more fun. I think to use FIAS sementic serach for extraction relavent documents ffrom the model and then provide the text to the model for proper human answer.

In [ ]:
from datasets import load_dataset
import datasets
from transformers import T5TokenizerFast
import pandas as pd

In [ ]:
dataset = load_dataset("eli5")
dataset

DatasetDict({
    train_eli5: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 272634
    })
    validation_eli5: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 9812
    })
    test_eli5: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 24512
    })
    train_asks: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 131778
    })
    validation_asks: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 2281
    })
    test_asks: Dataset({
       

lets combine the dataset as we only need to answer the question on all the possible topic we can have

WE are not going to train our model on this dataset. Insted we need to use this information for our sementic search.


In [ ]:
dataset_titles = list (dataset.keys() )
dataset_titles

['train_eli5',
 'validation_eli5',
 'test_eli5',
 'train_asks',
 'validation_asks',
 'test_asks',
 'train_askh',
 'validation_askh',
 'test_askh']

In [ ]:
dataset['test_asks'][1]

{'q_id': '3ko13p',
 'title': 'In the latest Avengers movie, there is a "Nexus" hub which routes every packet of traffic on the internet. What is the closest thing we have to a core of the internet in real life?',
 'selftext': '',
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['cuz3k5a',
   'cuz2ili',
   'cuz3zwf',
   'cuz2kz8',
   'cuz3e8d',
   'cuz2y2h',
   'cuz3p8y',
   'cuz3mot',
   'cuzaunl',
   'cuztchh',
   'cuz2j6p'],
  'text': ["Hey, Network Engineer here! \n\n (please don't get mad at me mechanical/electrical/etc engineers, it's just a job title.)\n\nThat idea is actually the exact opposite of how the internet works. (funny enough Cisco, a major networking vendor, has a line of switches called Nexus.)\n\nStrictly speaking the internet routing is completely distributed.  The BGP protocol is how that routing table is updated and hundreds of thousands of people/companies update the table regularly.   Those updates include    the destination  information and ho

In [ ]:
dataset['test_asks'][:5]


{'q_id': ['1xa1l8', '3ko13p', '2jy57p', '1943xl', '98va8l'],
 'title': ["If the heart is just a muscle, why can't I voluntarily flex it, or move it in any way?",
  'In the latest Avengers movie, there is a "Nexus" hub which routes every packet of traffic on the internet. What is the closest thing we have to a core of the internet in real life?',
  "Can somebody give me an intuitive understanding of Sine, Cosine, Tangent and perhaps Cotangent, Secant, Cosecant in such a way I won't forget?",
  "I've seen a magnet being dropped down a copper pipe... what happens if you do the opposite?",
  'What is the del operator in the Schrodinger equation?'],
 'selftext': ["Oftentimes, when when I focus on my heartbeat, I start to freak out that I might accidentally tell it to stop, or think something irrational like I could make it start beating manually, and then be stuck forever having to concentrate to contract my own heart to keep from dying. \n\nIf the heart is just a basic muscle, why can't I 

In [ ]:
temp_ds = pd.DataFrame([])



In [ ]:
temp_ds = dataset['test_asks'].to_pandas()
temp_ds

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls
0,1xa1l8,"If the heart is just a muscle, why can't I vol...","Oftentimes, when when I focus on my heartbeat,...",,askscience,"{'a_id': ['cf9gyt1', 'cf9h410', 'cf9i6b2'], 't...",{'url': []},{'url': []},{'url': ['http://en.wikipedia.org/wiki/Cardiac...
1,3ko13p,"In the latest Avengers movie, there is a ""Nexu...",,,askscience,"{'a_id': ['cuz3k5a', 'cuz2ili', 'cuz3zwf', 'cu...",{'url': []},{'url': []},{'url': ['http://www.infoworld.com/article/261...
2,2jy57p,Can somebody give me an intuitive understandin...,I use them all the time for equations but they...,,askscience,"{'a_id': ['clg9vcq', 'clga65f', 'clga5ty', 'cl...",{'url': []},{'url': []},{'url': ['https://en.wikipedia.org/wiki/Unit_c...
3,1943xl,I've seen a magnet being dropped down a copper...,"If you had a magnetic tube, and dropped a piec...",,askscience,"{'a_id': ['c8knyk4', 'c8kphpb', 'c8klvyd', 'c8...",{'url': []},{'url': []},{'url': ['http://en.wikipedia.org/wiki/Moving_...
4,98va8l,What is the del operator in the Schrodinger eq...,,,askscience,"{'a_id': ['e4j02cf', 'e4jcydl', 'e4j8usk'], 't...",{'url': []},{'url': []},{'url': []}
...,...,...,...,...,...,...,...,...,...
4457,3fbqr1,Is it possible to accurately predict the resul...,This is probably a really basic chemistry ques...,,askscience,"{'a_id': ['ctoqy36', 'ctonyz7'], 'text': ['You...",{'url': []},{'url': []},{'url': ['https://en.wikipedia.org/wiki/Chemic...
4458,2t0vxd,Is the light clock model wrong?,"Learning special relativity, teachers like to ...",,askscience,"{'a_id': ['cnuqju4'], 'text': ['The clock mode...",{'url': []},{'url': ['http://galileoandeinstein.physics.vi...,{'url': []}
4459,w74y9,What animal does this belong to?,_URL_0_\n\nI found this spine the other day an...,,askscience,"{'a_id': ['c5auwkc', 'c5aua11'], 'text': ['The...",{'url': []},{'url': ['http://imgur.com/a/FRVMv#0']},{'url': []}
4460,9r6yq1,Is fluid mechanics and vector analysis applica...,Just curious if there is any overlap into biol...,,askscience,"{'a_id': ['e8ifhna'], 'text': ['Momentum and e...",{'url': []},{'url': []},{'url': []}


In [ ]:
dataset['test_asks']

Dataset({
    features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
    num_rows: 4462
})

In [ ]:
temp_ds = pd.concat([temp_ds, dataset['train_asks'].to_pandas()], ignore_index=True)
temp_ds

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls
0,1xa1l8,"If the heart is just a muscle, why can't I vol...","Oftentimes, when when I focus on my heartbeat,...",,askscience,"{'a_id': ['cf9gyt1', 'cf9h410', 'cf9i6b2'], 't...",{'url': []},{'url': []},{'url': ['http://en.wikipedia.org/wiki/Cardiac...
1,3ko13p,"In the latest Avengers movie, there is a ""Nexu...",,,askscience,"{'a_id': ['cuz3k5a', 'cuz2ili', 'cuz3zwf', 'cu...",{'url': []},{'url': []},{'url': ['http://www.infoworld.com/article/261...
2,2jy57p,Can somebody give me an intuitive understandin...,I use them all the time for equations but they...,,askscience,"{'a_id': ['clg9vcq', 'clga65f', 'clga5ty', 'cl...",{'url': []},{'url': []},{'url': ['https://en.wikipedia.org/wiki/Unit_c...
3,1943xl,I've seen a magnet being dropped down a copper...,"If you had a magnetic tube, and dropped a piec...",,askscience,"{'a_id': ['c8knyk4', 'c8kphpb', 'c8klvyd', 'c8...",{'url': []},{'url': []},{'url': ['http://en.wikipedia.org/wiki/Moving_...
4,98va8l,What is the del operator in the Schrodinger eq...,,,askscience,"{'a_id': ['e4j02cf', 'e4jcydl', 'e4j8usk'], 't...",{'url': []},{'url': []},{'url': []}
...,...,...,...,...,...,...,...,...,...
136235,c3yf3z,"How much carbon dioxide is stored in leaves, t...",,,askscience,"{'a_id': ['erubgvz', 'ervj703', 'eru6zhw'], 't...",{'url': []},{'url': []},{'url': ['https://cdiac.ess-dive.lbl.gov/pns/c...
136236,q7db5,Why does cyanide kill you so quickly?,Hitler and many other German officials committ...,,askscience,"{'a_id': ['c3vc32o', 'c3vgpwl', 'c3vbhte', 'c3...",{'url': []},{'url': []},{'url': ['http://en.wikipedia.org/wiki/Death_o...
136237,163s6a,How does cyanide kill you?,I've been researching for a school paper but c...,,askscience,"{'a_id': ['c7sgpd7', 'c7sginx'], 'text': ['Cya...",{'url': []},{'url': []},{'url': ['http://www.jbc.org/content/269/39/24...
136238,itwn2,Why do LCD monitors have fixed refresh rates?,Why wouldn't an lcd monitor just be able to dr...,,askscience,"{'a_id': ['c26mrg1', 'c26nnpw'], 'text': ['Thi...",{'url': []},{'url': []},{'url': []}


In [ ]:
df = pd.DataFrame([])

for i in list (dataset.keys() ):
  df = pd.concat([df, dataset[i].to_pandas()], ignore_index=True)

df

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls
0,1oy5tc,in football whats the point of wasting the fir...,,,explainlikeimfive,"{'a_id': ['ccwtgnz', 'ccwtmho', 'ccwt946', 'cc...",{'url': []},{'url': []},{'url': []}
1,2lojul,Why are different tiers (regular < mid < premi...,I've noticed that the difference in price betw...,,explainlikeimfive,"{'a_id': ['clwqgxl', 'clwqpjq', 'clwuh3s'], 't...",{'url': []},{'url': []},{'url': []}
2,8v5e3s,Stars and Visibility,Why do stars in the night's sky seem to disapp...,,explainlikeimfive,"{'a_id': ['e1kpw6u'], 'text': ['It's a quirk o...",{'url': []},{'url': []},{'url': []}
3,1v3wij,How do we know all the money the government is...,We hear about these large billion dollar bank ...,,explainlikeimfive,"{'a_id': ['ceohakd', 'ceoikhs', 'ceoji15', 'ce...",{'url': []},{'url': ['http://dealbook.nytimes.com/2014/01/...,{'url': ['http://www.reuters.com/article/2013/...
4,2jlp6f,What are good and bad sides of manual and auto...,Please consider I'm not a driver. Automatic se...,,explainlikeimfive,"{'a_id': ['clcur3j'], 'text': ['Automatics wei...",{'url': []},{'url': []},{'url': []}
...,...,...,...,...,...,...,...,...,...
558664,1cwijw,What was the biggest man-made loss of life in ...,,,AskHistorians,"{'a_id': ['c9kne6c'], 'text': ['The largest I ...",{'url': []},{'url': []},{'url': []}
558665,8fucgx,"Hi, I was hoping that someone would be able to...",,,AskHistorians,"{'a_id': ['dy6lps1'], 'text': ['In Western Eur...",{'url': []},{'url': []},{'url': []}
558666,u4oom,What was the US Government's reaction to the D...,Obviously now there is an infrastructure in pl...,,AskHistorians,"{'a_id': ['c4sasnw'], 'text': ['The New Dealer...",{'url': []},{'url': []},{'url': []}
558667,bh9auu,What did America do after the dust bowl,"Today I was learning about the dust bowl, whic...",,AskHistorians,"{'a_id': ['elrx4pm'], 'text': ['Agricultural p...",{'url': []},{'url': []},{'url': []}


In [ ]:

# Lets check if the length of the dataframe is equal to the sum of the length of the individual datasets
len(df) == sum( [len(dataset[i]) for i in list (dataset.keys())] )

True

In [ ]:
df['answers'][0]

{'a_id': array(['ccwtgnz', 'ccwtmho', 'ccwt946', 'ccwvj0u'], dtype=object),
 'text': array(["Keep the defense honest, get a feel for the pass rush, open up the passing game. An offense that's too one dimensional will fail. And those rushes up the middle can be busted wide open sometimes for big yardage.",
        "If you throw the ball all the time, then the defense will adapt to always cover for a pass.  By doing a simple running play every now and then, you force the defense to stay close and guard against the run.  Sometimes, the offense can catch the defense off guard by faking a run and freeing up their receivers.\n\nAlso, you don't have to gain massive yards on every single play.  Sometimes, it works best to gain a few yards at a time.  As long as you get the first down, you are in good shape.",
        'In most cases the O-Line is supposed to make a hole for the running back to go through. If you run too many plays to the outside/throws the defense will catch on.\n\nAlso, 2 5 ya

In [ ]:
df['answers'][0].get ('text', None)

array(["Keep the defense honest, get a feel for the pass rush, open up the passing game. An offense that's too one dimensional will fail. And those rushes up the middle can be busted wide open sometimes for big yardage.",
       "If you throw the ball all the time, then the defense will adapt to always cover for a pass.  By doing a simple running play every now and then, you force the defense to stay close and guard against the run.  Sometimes, the offense can catch the defense off guard by faking a run and freeing up their receivers.\n\nAlso, you don't have to gain massive yards on every single play.  Sometimes, it works best to gain a few yards at a time.  As long as you get the first down, you are in good shape.",
       'In most cases the O-Line is supposed to make a hole for the running back to go through. If you run too many plays to the outside/throws the defense will catch on.\n\nAlso, 2 5 yard plays gets you a new set of downs.',
       "I you don't like those type of plays, w

In [ ]:
import pandas as pd

# Sample DataFrame
data = {
    'answers': [
        {'text': 'Value 1', 'key2': 'Another Value'},
        {'text': 'Value 2', 'key2': 'More Value'},
        # Add more rows with dictionaries
    ]
}

df1 = pd.DataFrame(data)

# Extract 'text' values from 'answers' and create a new column 'new_column'
df1['new_column'] = df['answers'].apply(lambda x: x.get('text', None))

# Display the updated DataFrame
df1


,answers,new_column
0,"{'text': 'Value 1', 'key2': 'Another Value'}","[Keep the defense honest, get a feel for the p..."
1,"{'text': 'Value 2', 'key2': 'More Value'}","[As someone who uses quality Premium, I wish t..."


In [ ]:
df['text_ans'] = df['answers'].apply(lambda x: x.get('text', None))
df

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls,text_ans
0,1oy5tc,in football whats the point of wasting the fir...,,,explainlikeimfive,"{'a_id': ['ccwtgnz', 'ccwtmho', 'ccwt946', 'cc...",{'url': []},{'url': []},{'url': []},"[Keep the defense honest, get a feel for the p..."
1,2lojul,Why are different tiers (regular < mid < premi...,I've noticed that the difference in price betw...,,explainlikeimfive,"{'a_id': ['clwqgxl', 'clwqpjq', 'clwuh3s'], 't...",{'url': []},{'url': []},{'url': []},"[As someone who uses quality Premium, I wish t..."
2,8v5e3s,Stars and Visibility,Why do stars in the night's sky seem to disapp...,,explainlikeimfive,"{'a_id': ['e1kpw6u'], 'text': ['It's a quirk o...",{'url': []},{'url': []},{'url': []},[It's a quirk of the human eye. At the center ...
3,1v3wij,How do we know all the money the government is...,We hear about these large billion dollar bank ...,,explainlikeimfive,"{'a_id': ['ceohakd', 'ceoikhs', 'ceoji15', 'ce...",{'url': []},{'url': ['http://dealbook.nytimes.com/2014/01/...,{'url': ['http://www.reuters.com/article/2013/...,[I'm pretty confident most of it isn't going b...
4,2jlp6f,What are good and bad sides of manual and auto...,Please consider I'm not a driver. Automatic se...,,explainlikeimfive,"{'a_id': ['clcur3j'], 'text': ['Automatics wei...",{'url': []},{'url': []},{'url': []},"[Automatics weigh more, so that alone makes ga..."
...,...,...,...,...,...,...,...,...,...,...
558664,1cwijw,What was the biggest man-made loss of life in ...,,,AskHistorians,"{'a_id': ['c9kne6c'], 'text': ['The largest I ...",{'url': []},{'url': []},{'url': []},[The largest I can think of is the Firebombing...
558665,8fucgx,"Hi, I was hoping that someone would be able to...",,,AskHistorians,"{'a_id': ['dy6lps1'], 'text': ['In Western Eur...",{'url': []},{'url': []},{'url': []},[In Western Europe there are very few specific...
558666,u4oom,What was the US Government's reaction to the D...,Obviously now there is an infrastructure in pl...,,AskHistorians,"{'a_id': ['c4sasnw'], 'text': ['The New Dealer...",{'url': []},{'url': []},{'url': []},[The New Dealers generally to a regional appro...
558667,bh9auu,What did America do after the dust bowl,"Today I was learning about the dust bowl, whic...",,AskHistorians,"{'a_id': ['elrx4pm'], 'text': ['Agricultural p...",{'url': []},{'url': []},{'url': []},[Agricultural practices changed in part by nec...


In [ ]:
df['text_ans'][0]

array(["Keep the defense honest, get a feel for the pass rush, open up the passing game. An offense that's too one dimensional will fail. And those rushes up the middle can be busted wide open sometimes for big yardage.",
       "If you throw the ball all the time, then the defense will adapt to always cover for a pass.  By doing a simple running play every now and then, you force the defense to stay close and guard against the run.  Sometimes, the offense can catch the defense off guard by faking a run and freeing up their receivers.\n\nAlso, you don't have to gain massive yards on every single play.  Sometimes, it works best to gain a few yards at a time.  As long as you get the first down, you are in good shape.",
       'In most cases the O-Line is supposed to make a hole for the running back to go through. If you run too many plays to the outside/throws the defense will catch on.\n\nAlso, 2 5 yard plays gets you a new set of downs.',
       "I you don't like those type of plays, w

In [ ]:
df = df.explode('text_ans', ignore_index=True)

In [ ]:
df['text_ans']

0          Keep the defense honest, get a feel for the pa...
1          If you throw the ball all the time, then the d...
2          In most cases the O-Line is supposed to make a...
3          I you don't like those type of plays, watch CF...
4          As someone who uses quality Premium, I wish th...
                                 ...                        
1442899    The largest I can think of is the Firebombing ...
1442900    In Western Europe there are very few specific ...
1442901    The New Dealers generally to a regional approa...
1442902    Agricultural practices changed in part by nece...
1442903    Timothy Mitchell's book *Rule of Experts* has ...
Name: text_ans, Length: 1442904, dtype: object

In [ ]:
df.head()

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls,text_ans
0,1oy5tc,in football whats the point of wasting the fir...,,,explainlikeimfive,"{'a_id': ['ccwtgnz', 'ccwtmho', 'ccwt946', 'cc...",{'url': []},{'url': []},{'url': []},"Keep the defense honest, get a feel for the pa..."
1,1oy5tc,in football whats the point of wasting the fir...,,,explainlikeimfive,"{'a_id': ['ccwtgnz', 'ccwtmho', 'ccwt946', 'cc...",{'url': []},{'url': []},{'url': []},"If you throw the ball all the time, then the d..."
2,1oy5tc,in football whats the point of wasting the fir...,,,explainlikeimfive,"{'a_id': ['ccwtgnz', 'ccwtmho', 'ccwt946', 'cc...",{'url': []},{'url': []},{'url': []},In most cases the O-Line is supposed to make a...
3,1oy5tc,in football whats the point of wasting the fir...,,,explainlikeimfive,"{'a_id': ['ccwtgnz', 'ccwtmho', 'ccwt946', 'cc...",{'url': []},{'url': []},{'url': []},"I you don't like those type of plays, watch CF..."
4,2lojul,Why are different tiers (regular < mid < premi...,I've noticed that the difference in price betw...,,explainlikeimfive,"{'a_id': ['clwqgxl', 'clwqpjq', 'clwuh3s'], 't...",{'url': []},{'url': []},{'url': []},"As someone who uses quality Premium, I wish th..."


In [ ]:
df['text_ans']

0          Keep the defense honest, get a feel for the pa...
1          If you throw the ball all the time, then the d...
2          In most cases the O-Line is supposed to make a...
3          I you don't like those type of plays, watch CF...
4          As someone who uses quality Premium, I wish th...
                                 ...                        
1442899    The largest I can think of is the Firebombing ...
1442900    In Western Europe there are very few specific ...
1442901    The New Dealers generally to a regional approa...
1442902    Agricultural practices changed in part by nece...
1442903    Timothy Mitchell's book *Rule of Experts* has ...
Name: text_ans, Length: 1442904, dtype: object

In [ ]:
df['text_ans'][0:5]

0    Keep the defense honest, get a feel for the pa...
1    If you throw the ball all the time, then the d...
2    In most cases the O-Line is supposed to make a...
3    I you don't like those type of plays, watch CF...
4    As someone who uses quality Premium, I wish th...
Name: text_ans, dtype: object

As we have seperated each answers isolated into new colmns. Lets convert it to datasets again and then tokenize it. 

In [ ]:
# colomn names in df
df.columns

Index(['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers',
       'title_urls', 'selftext_urls', 'answers_urls', 'text_ans'],
      dtype='object')

In [ ]:
# Lets drop un used colomns
df_final = df.drop(['q_id', 'selftext', 'document', 'subreddit', 'answers','title_urls', 'selftext_urls',], axis=1)
df_final

,title,answers_urls,text_ans
0,in football whats the point of wasting the fir...,{'url': []},"Keep the defense honest, get a feel for the pa..."
1,in football whats the point of wasting the fir...,{'url': []},"If you throw the ball all the time, then the d..."
2,in football whats the point of wasting the fir...,{'url': []},In most cases the O-Line is supposed to make a...
3,in football whats the point of wasting the fir...,{'url': []},"I you don't like those type of plays, watch CF..."
4,Why are different tiers (regular < mid < premi...,{'url': []},"As someone who uses quality Premium, I wish th..."
...,...,...,...
1442899,What was the biggest man-made loss of life in ...,{'url': []},The largest I can think of is the Firebombing ...
1442900,"Hi, I was hoping that someone would be able to...",{'url': []},In Western Europe there are very few specific ...
1442901,What was the US Government's reaction to the D...,{'url': []},The New Dealers generally to a regional approa...
1442902,What did America do after the dust bowl,{'url': []},Agricultural practices changed in part by nece...


In [ ]:
# lets convert this df to datasets
dataset_form_df = datasets.Dataset.from_pandas (df_final)
dataset_form_df

Dataset({
    features: ['title', 'answers_urls', 'text_ans'],
    num_rows: 1442904
})

In [ ]:
dataset_text = dataset_form_df.remove_columns(['answers_urls'])
dataset_text

Dataset({
    features: ['title', 'text_ans'],
    num_rows: 1442904
})

In [ ]:
def combine_text_and_title (row):
  return row['title'] + '\n' + row['text_ans']

In [ ]:
ds_comb = dataset_text.map( lambda x: {'text' : combine_text_and_title ( x ) } , batched=False, remove_columns= dataset_text.column_names)
ds_comb

Map:   0%|          | 0/1442904 [00:00<?, ? examples/s]

Map: 100%|██████████| 1442904/1442904 [01:23<00:00, 17292.02 examples/s]


Dataset({
    features: ['text'],
    num_rows: 1442904
})

In [ ]:
ds_comb[0]['text']

"in football whats the point of wasting the first two plays with a rush - up the middle - not regular rush plays i get those\nKeep the defense honest, get a feel for the pass rush, open up the passing game. An offense that's too one dimensional will fail. And those rushes up the middle can be busted wide open sometimes for big yardage."

In [ ]:
ds_comb.save_to_disk('eli5_dataset_title_text')

Saving the dataset (3/3 shards): 100%|██████████| 1442904/1442904 [00:00<00:00, 1705621.90 examples/s]


In [1]:
## Lets load the dataset form drive

In [2]:
import datasets
import huggingface_hub

In [13]:
ds_train = datasets.DatasetDict({'train': dataset})
ds_train.save_to_disk('eli5_dataset_title_text_train')


Saving the dataset (0/3 shards):   0%|          | 0/1442904 [00:00<?, ? examples/s]

In [14]:
ds_train = datasets.load_from_disk('eli5_dataset_title_text_train')
ds_train

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1442904
    })
})

In [8]:
import huggingface_hub 
import datasets
import os

In [12]:
import os

hugging_token = os.environ['HUGGINGFACE_TOKEN']

ds_train.push_to_hub('Safeer143/eli5_dataset_title_text', token=hugging_token)

Pushing dataset shards to the dataset hub:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/481 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/481 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/481 [00:00<?, ?ba/s]

Lets Tokenize the dataset

In [3]:
from transformers import AutoTokenizer, AutoModel
import datasets
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ckeckpoint = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(ckeckpoint)
model = AutoModel.from_pretrained(ckeckpoint)

model.to(device)

dataset_to_token = datasets.load_from_disk('eli5_dataset_title_text')

In [4]:
dataset_to_token

Dataset({
    features: ['text'],
    num_rows: 1442904
})

In [26]:
token = tokenizer('What is the meaning of life?', return_tensors='pt').to('cuda') 
model_ans = model(**token, return_dict=True)
model_ans.last_hidden_state.shape

torch.Size([1, 9, 768])

In [5]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [1]:
# this code is a mess. Its will break my computer
# token1 = tokenizer ( dataset_to_token['text'] , return_tensors='pt', padding=True, truncation=True) 
# token1

In [10]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt"  ).to(device)

In [11]:
token_1 = dataset_to_token.map(tokenize_function,  remove_columns=dataset_to_token.column_names)
token_1 

Map:   0%|          | 0/1442904 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1442904
})

In [53]:
token_1 = token_1.add_column(name='text', column= dataset_to_token['text'] )

In [55]:
token_1[0]

{'input_ids': [[101,
   1999,
   2374,
   2054,
   2015,
   1996,
   2391,
   1997,
   18313,
   1996,
   2034,
   2048,
   3248,
   2007,
   1037,
   5481,
   1011,
   2039,
   1996,
   2690,
   1011,
   2025,
   3180,
   5481,
   3248,
   1045,
   2131,
   2216,
   2562,
   1996,
   3639,
   7481,
   1010,
   2131,
   1037,
   2514,
   2005,
   1996,
   3413,
   5481,
   1010,
   2330,
   2039,
   1996,
   4458,
   2208,
   1012,
   2019,
   10048,
   2008,
   1005,
   1055,
   2205,
   2028,
   8789,
   2097,
   8246,
   1012,
   1998,
   2216,
   18545,
   2039,
   1996,
   2690,
   2064,
   2022,
   23142,
   2898,
   2330,
   2823,
   2005,
   2502,
   4220,
   4270,
   1012,
   102,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
 

In [16]:
len (token_1[0]['input_ids'] )

1

In [56]:
# token_1.remove_columns(['text'])
token_1.save_to_disk('eli5_dataset_title_text_tokenized_pt')

Saving the dataset (0/12 shards):   0%|          | 0/1442904 [00:00<?, ? examples/s]

In [1]:
# Lets load the tokenized dataset and get the model second last hidden state
from transformers import AutoTokenizer, AutoModel
import datasets
import torch

ckeckpoint = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(ckeckpoint)
model = AutoModel.from_pretrained(ckeckpoint)

load_dataset = datasets.load_from_disk('eli5_dataset_title_text_tokenized_pt')
load_dataset


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'text'],
    num_rows: 1442904
})

In [45]:
len (load_dataset[0]['input_ids'] )

512

In [58]:
load_dataset[0]

{'input_ids': [[101,
   1999,
   2374,
   2054,
   2015,
   1996,
   2391,
   1997,
   18313,
   1996,
   2034,
   2048,
   3248,
   2007,
   1037,
   5481,
   1011,
   2039,
   1996,
   2690,
   1011,
   2025,
   3180,
   5481,
   3248,
   1045,
   2131,
   2216,
   2562,
   1996,
   3639,
   7481,
   1010,
   2131,
   1037,
   2514,
   2005,
   1996,
   3413,
   5481,
   1010,
   2330,
   2039,
   1996,
   4458,
   2208,
   1012,
   2019,
   10048,
   2008,
   1005,
   1055,
   2205,
   2028,
   8789,
   2097,
   8246,
   1012,
   1998,
   2216,
   18545,
   2039,
   1996,
   2690,
   2064,
   2022,
   23142,
   2898,
   2330,
   2823,
   2005,
   2502,
   4220,
   4270,
   1012,
   102,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
 

In [59]:
torch.tensor(load_dataset[0]['input_ids'])

tensor([[  101,  1999,  2374,  2054,  2015,  1996,  2391,  1997, 18313,  1996,
          2034,  2048,  3248,  2007,  1037,  5481,  1011,  2039,  1996,  2690,
          1011,  2025,  3180,  5481,  3248,  1045,  2131,  2216,  2562,  1996,
          3639,  7481,  1010,  2131,  1037,  2514,  2005,  1996,  3413,  5481,
          1010,  2330,  2039,  1996,  4458,  2208,  1012,  2019, 10048,  2008,
          1005,  1055,  2205,  2028,  8789,  2097,  8246,  1012,  1998,  2216,
         18545,  2039,  1996,  2690,  2064,  2022, 23142,  2898,  2330,  2823,
          2005,  2502,  4220,  4270,  1012,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [61]:
model_ans = model(load_dataset[0])

TypeError: unhashable type: 'slice'

In [62]:
model_ds = load_dataset.remove_columns(['text'])
# model_ds.select(range(len(model_ds['input_ids']))).with_format('torch')
# model_ds
# model_ds = load_dataset

In [63]:
# Lets convert this dataset to tensor format
dataset_small = model_ds.select(range(100) )
dataset_small[0]

{'input_ids': [[101,
   1999,
   2374,
   2054,
   2015,
   1996,
   2391,
   1997,
   18313,
   1996,
   2034,
   2048,
   3248,
   2007,
   1037,
   5481,
   1011,
   2039,
   1996,
   2690,
   1011,
   2025,
   3180,
   5481,
   3248,
   1045,
   2131,
   2216,
   2562,
   1996,
   3639,
   7481,
   1010,
   2131,
   1037,
   2514,
   2005,
   1996,
   3413,
   5481,
   1010,
   2330,
   2039,
   1996,
   4458,
   2208,
   1012,
   2019,
   10048,
   2008,
   1005,
   1055,
   2205,
   2028,
   8789,
   2097,
   8246,
   1012,
   1998,
   2216,
   18545,
   2039,
   1996,
   2690,
   2064,
   2022,
   23142,
   2898,
   2330,
   2823,
   2005,
   2502,
   4220,
   4270,
   1012,
   102,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
 

In [64]:
def convert_to_tensor (row):
  return torch.tensor(row)

convert_to_tensor (dataset_small[1]['input_ids'])


tensor([[  101,  1999,  2374,  2054,  2015,  1996,  2391,  1997, 18313,  1996,
          2034,  2048,  3248,  2007,  1037,  5481,  1011,  2039,  1996,  2690,
          1011,  2025,  3180,  5481,  3248,  1045,  2131,  2216,  2065,  2017,
          5466,  1996,  3608,  2035,  1996,  2051,  1010,  2059,  1996,  3639,
          2097, 15581,  2000,  2467,  3104,  2005,  1037,  3413,  1012,  2011,
          2725,  1037,  3722,  2770,  2377,  2296,  2085,  1998,  2059,  1010,
          2017,  2486,  1996,  3639,  2000,  2994,  2485,  1998,  3457,  2114,
          1996,  2448,  1012,  2823,  1010,  1996, 10048,  2064,  4608,  1996,
          3639,  2125,  3457,  2011,  6904,  6834,  1037,  2448,  1998, 22198,
          2039,  2037, 19278,  1012,  2036,  1010,  2017,  2123,  1005,  1056,
          2031,  2000,  5114,  5294,  4210,  2006,  2296,  2309,  2377,  1012,
          2823,  1010,  2009,  2573,  2190,  2000,  5114,  1037,  2261,  4210,
          2012,  1037,  2051,  1012,  2004,  2146,  

In [116]:
def convert_ds_to_tensor (ds):
    for keys in ds.keys():
      ds[keys] = torch.tensor(ds[keys])
    return ds
  
convert_ds_to_tensor (dataset_small[1])  

{'input_ids': tensor([[  101,  1999,  2374,  2054,  2015,  1996,  2391,  1997, 18313,  1996,
           2034,  2048,  3248,  2007,  1037,  5481,  1011,  2039,  1996,  2690,
           1011,  2025,  3180,  5481,  3248,  1045,  2131,  2216,  2065,  2017,
           5466,  1996,  3608,  2035,  1996,  2051,  1010,  2059,  1996,  3639,
           2097, 15581,  2000,  2467,  3104,  2005,  1037,  3413,  1012,  2011,
           2725,  1037,  3722,  2770,  2377,  2296,  2085,  1998,  2059,  1010,
           2017,  2486,  1996,  3639,  2000,  2994,  2485,  1998,  3457,  2114,
           1996,  2448,  1012,  2823,  1010,  1996, 10048,  2064,  4608,  1996,
           3639,  2125,  3457,  2011,  6904,  6834,  1037,  2448,  1998, 22198,
           2039,  2037, 19278,  1012,  2036,  1010,  2017,  2123,  1005,  1056,
           2031,  2000,  5114,  5294,  4210,  2006,  2296,  2309,  2377,  1012,
           2823,  1010,  2009,  2573,  2190,  2000,  5114,  1037,  2261,  4210,
           2012,  1037,  20

In [105]:
ds_in_tensor = dataset_small.map (lambda x :  convert_ds_to_tensor(x))
ds_in_tensor 

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})

In [106]:
ds_in_tensor[0]

{'input_ids': [[101,
   1999,
   2374,
   2054,
   2015,
   1996,
   2391,
   1997,
   18313,
   1996,
   2034,
   2048,
   3248,
   2007,
   1037,
   5481,
   1011,
   2039,
   1996,
   2690,
   1011,
   2025,
   3180,
   5481,
   3248,
   1045,
   2131,
   2216,
   2562,
   1996,
   3639,
   7481,
   1010,
   2131,
   1037,
   2514,
   2005,
   1996,
   3413,
   5481,
   1010,
   2330,
   2039,
   1996,
   4458,
   2208,
   1012,
   2019,
   10048,
   2008,
   1005,
   1055,
   2205,
   2028,
   8789,
   2097,
   8246,
   1012,
   1998,
   2216,
   18545,
   2039,
   1996,
   2690,
   2064,
   2022,
   23142,
   2898,
   2330,
   2823,
   2005,
   2502,
   4220,
   4270,
   1012,
   102,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
 

In [102]:
def model_last_hidden_state (row):
  return model(**row)["last_hidden_state"][:,0].detach().cpu().numpy()[0]

model_out = model_last_hidden_state(ds_in_tensor[0])
model_out.shape

AttributeError: 'list' object has no attribute 'size'

In [93]:
dataset_small.column_names
dataset_small.select_columns(['input_ids'])

Dataset({
    features: ['input_ids'],
    num_rows: 100
})

In [117]:
# If I am performing these indevidual steps they are working fine. But when I am trying to combine them in a function it is not working
# the resaon I see is that when One function to convert ds to tensor is applied its not returning the tensor format when map is applied

def tokenize_from_model (ds):
    ds_val = ds
    print (ds_val.column_names)
    if len (ds_val.column_names) > 3:
      ds_val = ds.select_columns(['input_ids', 'token_type_ids', 'attention_mask'])
    print (ds_val)
    # tensor = dataset_small.map(convert_ds_to_tensor)
    tensor = convert_ds_to_tensor(dataset_small)
    print (tensor)
    # token = model_last_hidden_state(tensor)
    return tensor
    
# dataset_small.map(tokenize_from_model)
tokenize_from_model (dataset_small)

['input_ids', 'token_type_ids', 'attention_mask']
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})


AttributeError: 'Dataset' object has no attribute 'keys'

In [123]:
def token (ds):
    tenser = convert_ds_to_tensor(ds)
    token = model(**ds)["last_hidden_state"][:,0].detach().cpu().numpy()[0]
    return token

dataset_small.map(token)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

TypeError: Provided `function` which is applied to all elements of table returns a variable of type <class 'numpy.ndarray'>. Make sure provided `function` returns a variable of type `dict` (or a pyarrow table) to update the dataset or `None` if you are only interested in side effects.

In [22]:
model(** model_ds, return_dict=True)

NameError: name 'model_ds' is not defined

In [4]:
import datasets
# datasets.load_dataset ("Safeer143/marian-finetuned-kde4-en-to-fr-4-epochs-10000-samples")
dataset = datasets.load_from_disk ("eli5_dataset_title_text")
dataset

Dataset({
    features: ['text'],
    num_rows: 1442904
})

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
dataset.push_to_hub('eli5_dataset_title_text')

Pushing dataset shards to the dataset hub:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer, AutoModel
import datasets
import torch

ckeckpoint = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(ckeckpoint)
model = AutoModel.from_pretrained(ckeckpoint)

In [1]:
import datasets
dataset_online = datasets.load_dataset('Safeer143/eli5_dataset_title_text', split='train')

KeyboardInterrupt: 

In [1]:
from transformers import AutoTokenizer, AutoModel
import datasets
import torch

ckeckpoint = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(ckeckpoint)
model = AutoModel.from_pretrained(ckeckpoint)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

dataset = datasets.load_from_disk('eli5_dataset_title_text')
dataset

Dataset({
    features: ['text'],
    num_rows: 1442904
})

In [2]:
dataset_20000 = dataset.shuffle(52).select(range(20000) )
dataset_20000

Dataset({
    features: ['text'],
    num_rows: 20000
})

In [3]:
# lets tokenize this dataset and get the model last hidden state for fiass embeddings
def model_last_hidden_state (row):
  token = tokenizer(row['text'], return_tensors='pt', padding=True, truncation=True).to(device)

  return model(**token)["last_hidden_state"][:,0].detach().cpu().numpy()[0]
# model_last_hidden_state (dataset_100[0])

In [4]:
token_ds_20k = dataset_20000.map(lambda x : {'embeddings' : model_last_hidden_state(x) } )
token_ds_20k

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'embeddings'],
    num_rows: 20000
})

In [5]:
token_ds_20k[0]['embeddings']


[-0.2790078818798065,
 -0.45313718914985657,
 0.10761097073554993,
 0.14402496814727783,
 -1.12210214138031,
 0.08929474651813507,
 0.9192380309104919,
 1.004791021347046,
 0.10120101273059845,
 -0.10320691764354706,
 0.5131825804710388,
 -0.5070419907569885,
 -0.9077893495559692,
 0.9148262739181519,
 0.5832650065422058,
 -0.08860179036855698,
 -0.41548851132392883,
 0.2624037265777588,
 0.14002707600593567,
 0.14255189895629883,
 0.5409890413284302,
 0.11273255199193954,
 -0.04303742200136185,
 0.4311380684375763,
 0.08510442823171616,
 -0.12325286865234375,
 0.25632357597351074,
 -0.03572508692741394,
 0.2654544711112976,
 -0.23880396783351898,
 -0.41565513610839844,
 -0.010069145821034908,
 -0.3433397114276886,
 -0.6735960245132446,
 0.5280308127403259,
 -0.25775858759880066,
 0.16038917005062103,
 0.2434626966714859,
 -0.1035999208688736,
 0.5986385345458984,
 -0.9207701086997986,
 -0.24668370187282562,
 0.04567432776093483,
 -0.03752829134464264,
 -0.15332655608654022,
 -0.710654

In [6]:
token_ds_20k.save_to_disk('eli5_dataset_title_text_20k')


Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

In [8]:
import os 

hub_token = os.environ['HUGGINGFACE_TOKEN']

token_ds_20k.push_to_hub ('eli5_dataset_title_text_20k',token=hub_token)


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/20 [00:00<?, ?ba/s]

In [9]:
del token_ds_20k

In [10]:
torch.cuda.empty_cache()

### Lets add FIASS index to the dataset

In [5]:
import datasets
import torch 

device = torch.device('cuda')

dataset = datasets.load_from_disk('eli5_dataset_title_text_20k')
dataset

Dataset({
    features: ['text', 'embeddings'],
    num_rows: 20000
})

In [6]:
dataset_faiss = dataset.add_faiss_index (column='embeddings') 

  0%|          | 0/20 [00:00<?, ?it/s]

In [7]:
dataset_faiss[:10]

{'text': ["Paris attacks mega-thread\nWhy can't all the countries just 'gang up' and defeat isis once and for all?",
  'Buffalo buffalo Buffalo buffalo buffalo buffalo Buffalo buffalo.\nBuffalo bison [that other] Buffalo bison bully [in turn] bully [other] Buffalo bison.\n\n(The bison are all from Buffalo, New York.)',
  "How do scientists isolate chemicals like L-Dopa?\nMany chemicals in our bodies were discovered by the simplest mechanism: grind an organ or tissue to a pulp, then use different solvents to extract fractions of their content. Evaporate your extract and look for crystals, rinse, repeat. I see no reason why, with the appropriate procedure, you would not be able to extract L-Dopa from a dog's brain. That is, if you had access to the right organic solvents and aqueous buffers, and even then don't expect to obtain huge amounts each time. If this was viable we wouldn't bother to make it in the lab.\n\nAs for the last two questions, you basically ask to rebuild all chemistry 

In [45]:
from transformers import AutoTokenizer, AutoModel
model = AutoModel.from_pretrained('bert-base-uncased') 
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')  

In [46]:
question = 'What was student life like, what is the purpose of it?'
question

'What was student life like, what is the purpose of it?'

In [47]:
def question_to_token (question):
    return model(**tokenizer(question, return_tensors='pt',padding=True, truncation=True))["last_hidden_state"][:,0].detach().cpu().numpy()[0]

ques_tok = question_to_token (question)
print (question, ques_tok)


What was student life like, what is the purpose of it? [-1.78277977e-02  2.51936018e-01 -1.22532733e-02 -6.09233044e-03
 -2.54571319e-01 -1.20886832e-01  2.73525536e-01  5.46021700e-01
 -3.33344191e-02 -3.44985798e-02  4.39902723e-01 -1.98054507e-01
  9.65559706e-02  5.91033161e-01  4.64469731e-01 -1.45425528e-01
 -1.26795813e-01  4.46205646e-01  3.98994178e-01  2.73568351e-02
 -1.39325634e-02  8.72146860e-02 -2.23822102e-01  9.22088698e-02
  2.79964924e-01  4.03894484e-02  2.50946909e-01 -4.70472246e-01
 -4.14746031e-02  5.60571179e-02 -1.36980906e-01 -1.57469630e-01
 -2.53383309e-01 -4.03500408e-01 -1.20234028e-01 -1.67186350e-01
 -1.53100677e-02  1.01770952e-01  2.87011623e-01  2.55892098e-01
 -4.88596410e-01 -3.60970467e-01 -1.37847885e-02  1.88354969e-01
 -1.05545580e-01 -4.31199074e-01 -2.60517287e+00 -2.81230304e-02
 -3.52365762e-01  3.79686803e-03  2.04772711e-01 -3.55083123e-03
  5.38758993e-01  4.37534541e-01  6.91633001e-02  2.31169194e-01
 -1.50666520e-01  4.68057066e-01  1

In [48]:
context = dataset_faiss.get_nearest_examples('embeddings', ques_tok, k=5)
context 

NearestExamplesResults(scores=array([32.262104, 32.544247, 35.758667, 36.590504, 37.06971 ],
      dtype=float32), examples={'text': ["what's up with the specific jargon and dialect of airline employees?\nAnd what is a cross-check, and why do they get completed so quickly?", 'The biological purpose of an "Adam\'s Apple," and why women seemingly don\'t have them\nAs a woman with a prominent Adam\'s apple. What does this mean?', 'why the similarities in Greek and Roman theology? Where did these religions come from and where did they go?\nWhere did they come from, cotton eyed Joe?', 'Is there any evidence of slave owners killing their slaves when slavery was abolished in the US?\nIf it happened, what would be the rationalization for doing that?', 'how come we get better at darts and billiards after a few drinks before it goes downhill?\nWhat about the come down effect of a small amount?'], 'embeddings': [[0.07101025432348251, 0.37527263164520264, 0.23619066178798676, 0.049828629940748215,

In [49]:
model_t5 = AutoModel.from_pretrained('t5-base')
tokenizer_t5 = AutoTokenizer.from_pretrained('t5-base')
''' 
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer_t5 = T5Tokenizer.from_pretrained("t5-small")
model_t5 = T5ForConditionalGeneration.from_pretrained("t5-small")
'''

'\nfrom transformers import T5Tokenizer, T5ForConditionalGeneration\n\ntokenizer_t5 = T5Tokenizer.from_pretrained("t5-small")\nmodel_t5 = T5ForConditionalGeneration.from_pretrained("t5-small")\n'

In [50]:
context[1]['text']

["what's up with the specific jargon and dialect of airline employees?\nAnd what is a cross-check, and why do they get completed so quickly?",
 'The biological purpose of an "Adam\'s Apple," and why women seemingly don\'t have them\nAs a woman with a prominent Adam\'s apple. What does this mean?',
 'why the similarities in Greek and Roman theology? Where did these religions come from and where did they go?\nWhere did they come from, cotton eyed Joe?',
 'Is there any evidence of slave owners killing their slaves when slavery was abolished in the US?\nIf it happened, what would be the rationalization for doing that?',
 'how come we get better at darts and billiards after a few drinks before it goes downhill?\nWhat about the come down effect of a small amount?']

In [51]:
def get_context(question):
    quest_tok = model(**tokenizer(question, return_tensors='pt',padding=True, truncation=True))["last_hidden_state"][:,0].detach().cpu().numpy()[0]

    context = dataset_faiss.get_nearest_examples('embeddings', quest_tok, k=5)

    return context[1]['text']

get_context(question)

["what's up with the specific jargon and dialect of airline employees?\nAnd what is a cross-check, and why do they get completed so quickly?",
 'The biological purpose of an "Adam\'s Apple," and why women seemingly don\'t have them\nAs a woman with a prominent Adam\'s apple. What does this mean?',
 'why the similarities in Greek and Roman theology? Where did these religions come from and where did they go?\nWhere did they come from, cotton eyed Joe?',
 'Is there any evidence of slave owners killing their slaves when slavery was abolished in the US?\nIf it happened, what would be the rationalization for doing that?',
 'how come we get better at darts and billiards after a few drinks before it goes downhill?\nWhat about the come down effect of a small amount?']

In [52]:
question = 'What is the meaning of life? How to live happy in this short life?'

context = get_context(question)

text = f"question: {question} context: {context}"
text


'question: What is the meaning of life? How to live happy in this short life? context: ["what\'s up with the specific jargon and dialect of airline employees?\\nAnd what is a cross-check, and why do they get completed so quickly?", \'Why incest is bad\\nNo one is mentioning the severe psychological damage incestuous relationships can have on a child?\', \'Why is it acceptable for governments to kill other people during war without being punished?\\nIt is not acceptable. Who is going to stop them?\', \'How do Stingray towers work, and should I be worried that a friend found one near me?\\nHow do they get them, who makes them?\', \'What is happening when I "zone out"?\\nAnd why does it happen more when I\\\'m tired?\']'

In [53]:
input_t5 = tokenizer_t5 (text, return_tensors='pt', padding=True, truncation=True).convert_to_tensors('pt')
input_t5 

{'input_ids': tensor([[  822,    10,   363,    19,     8,  2530,    13,   280,    58,   571,
            12,   619,  1095,    16,    48,   710,   280,    58,  2625,    10,
           784,   121,  9170,    31,     7,    95,    28,     8,   806,     3,
          5670,  5307,    11, 28461,    13, 11545,  1652,    58,     2,    29,
          7175,   125,    19,     3,     9,  2269,    18, 10031,     6,    11,
           572,   103,    79,   129,  2012,    78,  1224,  4609,     6,     3,
            31, 17891,    16,    75,   222,    19,  1282,     2,    29,  4168,
            80,    19,     3, 27492,     8,  5274, 11041,  1783,    16,    75,
           222, 13281,  3079,    54,    43,    30,     3,     9,   861,    58,
            31,     6,     3,    31, 17891,    19,    34,  9961,    21, 10524,
            12,  5781,   119,   151,   383,   615,   406,   271, 31171,    58,
             2,    29,   196,    17,    19,    59,  9961,     5,  2645,    19,
           352,    12,  1190,   135,  

In [55]:
outputs = model_t5(**input_t5, decoder_input_ids=input_t5["input_ids"])
outputs 

Seq2SeqModelOutput(last_hidden_state=tensor([[[ 0.1930, -0.2784,  0.3673,  ...,  0.0551, -0.0867, -0.0715],
         [ 0.2662, -0.2742,  0.2871,  ...,  0.1022, -0.0239, -0.0505],
         [ 0.1681, -0.1734,  0.2278,  ..., -0.1793, -0.0292, -0.0994],
         ...,
         [ 0.1789, -0.3086,  0.1156,  ...,  0.0994,  0.0078,  0.0013],
         [ 0.2151, -0.2806,  0.0786,  ...,  0.1124, -0.0045,  0.0064],
         [ 0.2092, -0.3186,  0.0834,  ...,  0.1143,  0.0061,  0.0448]]],
       grad_fn=<MulBackward0>), past_key_values=((tensor([[[[-8.8160e-01,  2.8566e-01, -1.2567e+00,  ...,  3.0997e-01,
            6.8413e-01, -1.1682e+00],
          [ 7.2569e-01,  9.7695e-01, -1.4260e-01,  ...,  3.4269e-01,
            4.8386e-01, -1.1693e+00],
          [ 5.2962e-01, -2.5660e-01, -2.8361e+00,  ...,  3.9165e-01,
            8.1875e-01, -1.6981e+00],
          ...,
          [ 4.2566e-01,  4.3054e-01, -8.6227e-01,  ...,  7.1462e-01,
           -5.7628e-01, -4.6562e-01],
          [-2.6895e-01,  6.9

In [58]:
generated_text = tokenizer_t5.decoder (outputs, skip_special_tokens=True)
generated_text

TypeError: 'tokenizers.decoders.Metaspace' object is not callable

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer5 = T5Tokenizer.from_pretrained("t5-small")
model5 = T5ForConditionalGeneration.from_pretrained("t5-small")


task_prefix = "translate English to German: "
# use different length sentences to test batching
sentences = ["The house is wonderful.", "I like to work in NYC."]

inputs = tokenizer5([task_prefix + sentence for sentence in sentences], return_tensors="pt", padding=True)

output_sequences = model5.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=False,  # disable sampling to test if batching affects output
)

print(tokenizer5.batch_decode(output_sequences, skip_special_tokens=True))
output_sequences


['Das Haus ist wunderbar.', 'Ich arbeite gerne in NYC.']


C:\Users\Safeer\AppData\Roaming\Python\Python38\site-packages\transformers\generation\utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[    0,   644,  4598,   229, 19250,     5,     1,     0,     0,     0],
        [    0,  1674,     3,  5269,    15,  5462,    16, 13465,     5,     1]])

In [59]:
import datasets
import torch 

device = torch.device('cuda')

dataset = datasets.load_from_disk('eli5_dataset_title_text_20k')
dataset

dataset_faiss = dataset.add_faiss_index (column='embeddings')

from transformers import AutoTokenizer, AutoModel
model = AutoModel.from_pretrained('bert-base-uncased') 
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')  

question = 'What was student life like, what is the purpose of it?'

def question_to_token (question):
    return model(**tokenizer(question, return_tensors='pt',padding=True, truncation=True))["last_hidden_state"][:,0].detach().cpu().numpy()[0]

ques_tok = question_to_token (question)

context = dataset_faiss.get_nearest_examples('embeddings', ques_tok, k=5)

model_t5 = AutoModel.from_pretrained('t5-base')
tokenizer_t5 = AutoTokenizer.from_pretrained('t5-base')

def get_context(question):
    quest_tok = model(**tokenizer(question, return_tensors='pt',padding=True, truncation=True))["last_hidden_state"][:,0].detach().cpu().numpy()[0]

    context = dataset_faiss.get_nearest_examples('embeddings', quest_tok, k=5)

    return context[1]['text']

question = 'What is the meaning of life? How to live happy in this short life?'

context = get_context(question)

text = f"question: {question} context: {context}"

input_t5 = tokenizer_t5 (text, return_tensors='pt', padding=True, truncation=True).convert_to_tensors('pt')

outputs = model_t5(**input_t5, decoder_input_ids=input_t5["input_ids"])

generated_text = tokenizer_t5.decoder (outputs, skip_special_tokens=True)

generated_text

  0%|          | 0/20 [00:00<?, ?it/s]

C:\Users\Safeer\AppData\Roaming\Python\Python38\site-packages\transformers\models\t5\tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


TypeError: 'tokenizers.decoders.Metaspace' object is not callable

In [60]:
import datasets
import torch
from transformers import AutoTokenizer, AutoModel

device = torch.device('cuda')

dataset = datasets.load_from_disk('eli5_dataset_title_text_20k')
dataset_faiss = dataset.add_faiss_index(column='embeddings')

model = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

model_t5 = AutoModel.from_pretrained('t5-base')
tokenizer_t5 = AutoTokenizer.from_pretrained('t5-base')

def question_to_token(question):
    return model(**tokenizer(question, return_tensors='pt', padding=True, truncation=True))["last_hidden_state"][:, 0].detach().cpu().numpy()[0]

def get_context(question):
    quest_tok = model_t5(**tokenizer_t5(question, return_tensors='pt', padding=True, truncation=True))["last_hidden_state"][:, 0].detach().cpu().numpy()[0]
    context = dataset_faiss.get_nearest_examples('embeddings', quest_tok, k=5)
    return context[1]['text']

question = 'What is the meaning of life? How to live happy in this short life?'

context = get_context(question)

text = f"question: {question} context: {context}"

input_t5 = tokenizer_t5(text, return_tensors='pt', padding=True, truncation=True)

outputs = model_t5.generate(**input_t5)

generated_text = tokenizer_t5.decode(outputs[0], skip_special_tokens=True)

print(generated_text)


  0%|          | 0/20 [00:00<?, ?it/s]

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

In [62]:
import datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load your dataset and set up your Faiss index as before

# Load the T5 tokenizer and model
model_t5 = AutoModelForSeq2SeqLM.from_pretrained('t5-base')
tokenizer_t5 = AutoTokenizer.from_pretrained('t5-base')

def generate_answer(question, context):
    input_text = f"question: {question} context: {context}"
    inputs = tokenizer_t5(input_text, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs['input_ids']

    # Generate an answer
    answer_ids = model_t5.generate(input_ids,  decoder_input_ids=input_t5["input_ids"])
    generated_answer = tokenizer_t5.decode(answer_ids[0], skip_special_tokens=True)

    return generated_answer

# Provide a question and context
question = 'What is the meaning of life? How to live happy in this short life?'
context = get_context(question)  # You can use your `get_context` function

# Generate the answer
generated_answer = generate_answer(question, context)
print("Generated Answer:", generated_answer)


ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds